In [1]:
#import OpenCV module
import cv2
#import os module for reading training data directories and paths
import os
#import numpy to convert python lists to numpy arrays as 
#it is needed by OpenCV face recognizers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from PIL import Image
from numpy import * 
import tensorflow as tf


from skimage import color
from skimage import io
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger

In [2]:
def prepare_training_data(data_folder_path):
    
    #------STEP-1--------
    #get the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)
    
    #list to hold all subject faces
    faces = []
    #list to hold labels for all subjects
    labels = []
    
    #let's go through each directory and read images within it
    for dir_name in dirs:
        
        #our subject directories start with letter 's' so
        #ignore any non-relevant directories if any
        if not dir_name.startswith("s"):
            continue;
            
        #------STEP-2--------
        #extract label number of subject from dir_name
        #format of dir name = slabel
        #, so removing letter 's' from dir_name will give us label
        label = int(dir_name.replace("s", ""))
        
        #build path of directory containin images for current subject subject
        #sample subject_dir_path = "training-data/s1"
        subject_dir_path = data_folder_path + "/" + dir_name
        
        #get the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)
        
        #------STEP-3--------
        #go through each image name, read image, 
        #detect face and add face to list of faces
        for image_name in subject_images_names:
            
            #ignore system files like .DS_Store
            if image_name.startswith("."):
                continue;
            
            #build image path
            #sample image path = training-data/s1/1.pgm
            image_path = subject_dir_path + "/" + image_name

            #read image
            #temp = cv2.imread(image_path)
           # temp = Image.open(image_path)
            img = io.imread(image_path)
            # imgGray = color.rgb2gray(img)
            #temp=Image.open('a.jpg') 
            img = img.astype('float32') / 255.0
           
                     
            #------STEP-4--------
            #for the purpose of this tutorial
            #we will ignore faces that are not detected
            if img is not None:
                #add face to list of faces
                
                faces.append(img)
#                 print(img.shape)
#                 print(label)
                #add label for this face
                labels.append(label)
              
    
    
    # print(labels)

    return faces, labels
            
           

In [3]:
print("Preparing data...")
facestr, labelstr = prepare_training_data('C:\\Users\\minnu\\Downloads\\Face-Recognition\\Face-Recognition-master\\Datasets\\Faces\\training')
print("Data prepared")

faceste, labelste = prepare_training_data('C:\\Users\\minnu\\Downloads\\Face-Recognition\\Face-Recognition-master\\Datasets\\Faces\\testing')
print("Data prepared")
facesval, labelsval = prepare_training_data('C:\\Users\\minnu\\Downloads\\Face-Recognition\\Face-Recognition-master\\Datasets\\Faces\\validation')
print("Data prepared")

Preparing data...
Data prepared
Data prepared
Data prepared


In [4]:
x_train=np.array(facestr)
y_train=np.array(labelstr)
print(x_train.ndim)
print("train pics: ", x_train.shape)
print("trainlabels: ", y_train.shape)
x_test=np.array(faceste)
y_test=np.array(labelste)
print("test pics: ", x_test.shape)
print("test labels: ", y_test.shape)
x_val=np.array(facesval)
y_val=np.array(labelsval)
print("test pics: ", x_val.shape)
print("test labels: ", y_val.shape)

3
train pics:  (280, 80, 70)
trainlabels:  (280,)
test pics:  (80, 80, 70)
test labels:  (80,)
test pics:  (40, 80, 70)
test labels:  (40,)


In [5]:
list_names = []
list_accuracy = []

In [6]:
x_train = x_train.reshape(-1,80,70,1)
x_test = x_test.reshape(-1,80,70,1)
x_val = x_val.reshape(-1,80,70,1)
print("x_train: ",x_train.shape)
print("x_test: ",x_test.shape)

x_train:  (280, 80, 70, 1)
x_test:  (80, 80, 70, 1)


In [45]:
num_classes = max(max(y_train), max(y_test), max(y_val))+1
print(num_classes)

41


In [46]:
from tensorflow.python.keras.utils.np_utils import to_categorical # convert to one-hot-encoding

y_train_ = to_categorical(y_train, num_classes=num_classes)
y_test_ = to_categorical(y_test, num_classes=num_classes)
y_val_ = to_categorical(y_val, num_classes=num_classes)


print("y_train_ shape: ",y_train_.shape)
print("y_test_ shape: ",y_test_.shape)

print(y_train)
print('test lables')
print(y_test)
print(y_val)

y_train_ shape:  (280, 41)
y_test_ shape:  (80, 41)
[ 1  1  1  1  1  1  1 10 10 10 10 10 10 10 11 11 11 11 11 11 11 12 12 12
 12 12 12 12 13 13 13 13 13 13 13 14 14 14 14 14 14 14 15 15 15 15 15 15
 15 16 16 16 16 16 16 16 17 17 17 17 17 17 17 18 18 18 18 18 18 18 19 19
 19 19 19 19 19  2  2  2  2  2  2  2 20 20 20 20 20 20 20 21 21 21 21 21
 21 21 22 22 22 22 22 22 22 23 23 23 23 23 23 23 24 24 24 24 24 24 24 25
 25 25 25 25 25 25 26 26 26 26 26 26 26 27 27 27 27 27 27 27 28 28 28 28
 28 28 28 29 29 29 29 29 29 29  3  3  3  3  3  3  3 30 30 30 30 30 30 30
 31 31 31 31 31 31 31 32 32 32 32 32 32 32 33 33 33 33 33 33 33 34 34 34
 34 34 34 34 35 35 35 35 35 35 35 36 36 36 36 36 36 36 37 37 37 37 37 37
 37 38 38 38 38 38 38 38 39 39 39 39 39 39 39  4  4  4  4  4  4  4 40 40
 40 40 40 40 40  5  5  5  5  5  5  5  6  6  6  6  6  6  6  7  7  7  7  7
  7  7  8  8  8  8  8  8  8  9  9  9  9  9  9  9]
test lables
[ 1  1 10 10 11 11 12 12 13 13 14 14 15 15 16 16 17 17 18 18 19 19  2  2
 20 20 21 

In [47]:
from tensorflow.python.keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
face_model = Sequential()
face_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80, 70, 1)))
face_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))

face_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
face_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))

face_model.add(Flatten())
face_model.add(Dense(512, activation='relu'))
face_model.add(Dropout(0.5))
face_model.add(Dense(num_classes, activation='softmax'))

optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08)
face_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

output_dir = "C:\\Users\\minnu\\Downloads\\Face-Recognition\\Face-Recognition-master\\checkpoint"
csv_logger = CSVLogger(os.path.join(output_dir, 'training.log'))

checkpoint_filepath_val = os.path.join(output_dir, 'model_chkpt_val.h5')
model_checkpoint_val = ModelCheckpoint(filepath=checkpoint_filepath_val,
                                       save_weights_only=False,
                                       monitor='val_accuracy',
                                       mode='max',
                                       save_best_only=True,
                                       verbose=1)

checkpoint_filepath_acc = os.path.join(output_dir, 'model_chkpt_acc.h5')
model_checkpoint_acc = ModelCheckpoint(filepath=checkpoint_filepath_acc,
                                       save_weights_only=False,
                                       monitor='accuracy',
                                       mode='max',
                                       save_best_only=True,
                                       verbose=1)

learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=3,
    verbose=1,
    factor=0.7,
    min_lr=0.00000000001
)

epoch = 50
batch_size = 128

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=5,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.05, # Randomly zoom image 
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
datagen.fit(x_train)

history = face_model.fit(
    datagen.flow(x_train, y_train_, batch_size=batch_size),
    epochs=epoch,
    validation_data=(x_val, y_val_),
    verbose=2,
    steps_per_epoch=x_train.shape[0] // batch_size,
#     callbacks=[learning_rate_reduction,csv_logger, model_checkpoint_val, model_checkpoint_acc]
)

model_json = face_model.to_json()
with open("face_model.json", "w") as json_file:
    json_file.write(model_json)
    
face_model.save_weights('face_model.h5')
print("done")


Epoch 1/50
2/2 - 4s - loss: 4.1370 - accuracy: 0.0117 - val_loss: 3.7128 - val_accuracy: 0.0250 - 4s/epoch - 2s/step
Epoch 2/50
2/2 - 3s - loss: 3.7284 - accuracy: 0.0273 - val_loss: 3.7117 - val_accuracy: 0.0500 - 3s/epoch - 1s/step
Epoch 3/50
2/2 - 2s - loss: 3.7070 - accuracy: 0.0329 - val_loss: 3.7079 - val_accuracy: 0.0250 - 2s/epoch - 798ms/step
Epoch 4/50
2/2 - 2s - loss: 3.7137 - accuracy: 0.0197 - val_loss: 3.7062 - val_accuracy: 0.0250 - 2s/epoch - 795ms/step
Epoch 5/50
2/2 - 2s - loss: 3.7425 - accuracy: 0.0197 - val_loss: 3.7048 - val_accuracy: 0.0250 - 2s/epoch - 799ms/step
Epoch 6/50
2/2 - 3s - loss: 3.6986 - accuracy: 0.0195 - val_loss: 3.6989 - val_accuracy: 0.0250 - 3s/epoch - 1s/step
Epoch 7/50
2/2 - 2s - loss: 3.6947 - accuracy: 0.0263 - val_loss: 3.6957 - val_accuracy: 0.0250 - 2s/epoch - 840ms/step
Epoch 8/50
2/2 - 3s - loss: 3.7195 - accuracy: 0.0117 - val_loss: 3.6956 - val_accuracy: 0.0250 - 3s/epoch - 1s/step
Epoch 9/50
2/2 - 2s - loss: 3.7030 - accuracy: 0.019

In [48]:
model_json = face_model.to_json()
with open("face_model.json", "w") as json_file:
    json_file.write(model_json)

face_model.save_weights('face_model.h5')
print("Saved")

Saved


In [49]:
face_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 78, 68, 32)        320       
                                                                 
 conv2d_5 (Conv2D)           (None, 76, 66, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 38, 33, 64)        0         
 g2D)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 38, 33, 64)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 36, 31, 128)       73856     
                                                                 
 conv2d_7 (Conv2D)           (None, 34, 29, 128)       147584    
                                                      

In [50]:
print("evaluate in test data")
result=face_model.evaluate(x_test,y_test_,batch_size=64)
print("test loss,test acc",result)

evaluate in test data
2/2 [==============================] - 0s 44ms/step - loss: 0.3724 - accuracy: 0.9125
test loss,test acc [0.3723891079425812, 0.9125000238418579]


In [51]:

list_names=[]
#y_pred = model.predict_classes(x_test)
y_pred = np.argmax(face_model.predict(x_test), axis=-1)

y_test = y_test.reshape(-1,)

diff = y_test - y_pred
diff = diff.reshape(-1,1)

true = 0
for i in range(0,len(diff)):
    if diff[i] == 0:
        true = true + 1

Cnn_accuracy = round(100*true/len(diff),2)

print("Cnn_accuracy is %", Cnn_accuracy)

list_names.append("CNN")
list_accuracy.append(Cnn_accuracy)


3/3 [==============================] - 0s 58ms/step
Cnn_accuracy is % 91.25


In [52]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       1.00      0.50      0.67         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       1.00      0.50      0.67         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         2
          13       0.67      1.00      0.80         2
          14       1.00      1.00      1.00         2
          15       1.00      1.00      1.00         2
          16       0.67      1.00      0.80         2
          17       1.00      0.50      0.67         2
          18       1.00    

In [53]:
# # Assuming your model is named 'face_model'
# predict = face_model.predict(x_test)
# y_pred = np.argmax(predict, axis=-1)

# y_test = y_test.reshape(-1,)

# diff = y_test - y_pred
# diff = diff.reshape(-1, 1)

# true = 0
# precisions = []

# for i in range(0, len(diff)):
#     if diff[i] == 0:
#         true = true + 1
#     # Calculate precision for the specific image
#     precision = precision_score(y_test, y_pred, average='macro')
#     precisions.append(precision)

# cnn_accuracy = round(100 * true / len(diff), 2)

# print("CNN accuracy is %", cnn_accuracy)

# list_names.append("CNN")
# list_accuracy.append(cnn_accuracy)

# # Plot random images with true and predicted labels
# plt.figure(figsize=(80, 100))
# for i in range(40):  # Change the range based on the number of images you want to visualize
#     index = random.randint(0, len(x_test) - 1)
#     image = x_test[index].reshape(80, 70)  # Assuming grayscale images
#     true_label = y_test[index]
#     predicted_label = y_pred[index]

#     # Check if the prediction is correct
#     if true_label == predicted_label:
#         color = 'green'
#         accuracy_label = f'Correct\n'
#     else:
#         color = 'red'
#         accuracy_label = f'Incorrect\n'
#     accuracy_label_text = f'Test Accuracy: {cnn_accuracy}%\n'
#     precision_text = f'Precision: {precisions[index]:.2f}\n'

#     plt.subplot(8, 5, i + 1)
#     plt.imshow(image, cmap='gray')
#     plt.axis('off')

#     # Print labels below each image
#     plt.text(0.5, -0.5, f'True: {true_label}\nPredicted: {predicted_label}\n{accuracy_label}\n{accuracy_label_text}\n{precision_text}', color=color, fontsize=50, horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)

# # Increase the spacing between subplots (both horizontally and vertically)
# plt.subplots_adjust(wspace=0.4, hspace=0.8)

# plt.show()


In [69]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

def load_face_model(model_path):
    return load_model(model_path)

def preprocess_image(img_path):
    # Read the image
    img = cv2.imread(img_path)
    
    # Convert the image to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Resize the image to match the model's expected input shape (80, 70)
    img_resized = cv2.resize(img_gray, (80, 70))
    
    # Expand dimensions to match the model's input shape
    img_array = np.expand_dims(img_resized, axis=0)
    img_array = np.expand_dims(img_array, axis=-1)
    
    # Normalize the pixel values to be between 0 and 1
    img_array = img_array.astype('float32') / 255.0
    return img_array


def recognize_face_single_image(model, img_array):
    # Predict the face using the trained model
    face_prediction = model.predict(img_array)
    # Get the predicted label index
    print(face_prediction)
    predicted_label_index = np.argmax(face_prediction[0])
    predicted_label=y_test[predicted_label_index]
    acc=face_prediction[0][predicted_label_index]*100
    return predicted_label,acc

# # Load the trained face recognition model
# model_path = 'path/to/your/trained_model.h5'
# face_model = load_face_model(model_path)

# Provide the path to the image you want to test
image_path = r"C:\\Users\\minnu\\Downloads\\Face-Recognition\\Face-Recognition-master\\Datasets\\Faces\\testing\\s24\\231_24.jpg"
# image_path = r"D:\\Downloads\\larine.jpg"

# Preprocess the image
image_array = preprocess_image(image_path)

# Recognize the face in the single image
predicted_label,acc = recognize_face_single_image(face_model, image_array)
print(y_test)
# Print the predicted label index
print("Predicted Label :",predicted_label)
print("Predicted acc :",acc)



1/1 [==============================] - 0s 33ms/step
[[5.2430250e-06 3.4496931e-03 2.9988079e-03 1.3571486e-02 9.1347471e-03
  1.3409994e-02 2.2028308e-02 6.6727716e-03 2.9720394e-03 2.7042360e-03
  1.5409944e-02 2.5389681e-03 7.3620264e-04 1.6378105e-03 4.0347117e-04
  1.0787424e-02 4.7559354e-02 1.2591376e-02 4.6932586e-03 9.4502798e-04
  5.9168179e-02 3.7174169e-03 1.3410048e-02 1.7604882e-01 5.6057656e-03
  1.1010218e-02 2.6825248e-04 3.3871476e-03 1.4109131e-03 2.1517058e-01
  1.2510069e-01 8.3801404e-02 3.7546240e-02 2.1192124e-02 1.2008377e-02
  5.7687890e-03 7.9974849e-03 7.3085254e-04 3.3432685e-03 2.7271939e-02
  1.1791501e-02]]
[ 1  1 10 10 11 11 12 12 13 13 14 14 15 15 16 16 17 17 18 18 19 19  2  2
 20 20 21 21 22 22 23 23 24 24 25 25 26 26 27 27 28 28 29 29  3  3 30 30
 31 31 32 32 33 33 34 34 35 35 36 36 37 37 38 38 39 39  4  4 40 40  5  5
  6  6  7  7  8  8  9  9]
Predicted Label : 22
Predicted acc : 21.517057716846466


In [24]:
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image
# # Load JSON data from the file
# with open("labels_model.json", "r") as json_file:
#     labels_json = json_file.read()

# # Deserialize JSON to a dictionary
# class_indices_dict = json.loads(labels_json)

# # Convert dictionary values to an array
# class_indices_array = list(class_indices_dict)
# # Load the model structure from the saved json file
# with open("C:\\Users\\minnu\\Downloads\\Face-Recognition\\Face-Recognition-master\\face_model.json", "r") as json_file:
#     loaded_model_json = json_file.read()
#     face_model = model_from_json(loaded_model_json)

# Load the trained weights into the model
# face_model.load_weights('C:\\Users\\minnu\\Downloads\\Face-Recognition\\Face-Recognition-master\\checkpoint\\model_chkpt_val.h5')
# face_model.load_weights('face_model.h5')

# Load the image for detection
# img_path = r"C:\\Users\\minnu\\Downloads\\Face-Recognition\\Face-Recognition-master\\Datasets\\Faces\\testing\\s18\\171_18.jpg"
img_path = r"D:\\Downloads\\larine.jpg"


print(os.path.basename(img_path))

img = image.load_img(img_path, target_size=(80, 70), color_mode="grayscale")
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# print(img_array)
# img_array /= 255.0

# img_array = io.imread(image_path).reshape(-1,80,70,1)

# Make prediction
face_prediction = face_model.predict(img_array)
print(face_prediction)


# print(emotion_prediction[0][23]*100)

# Get the predicted emotion label
predicted_label_index = np.argmax(face_prediction[0])
accuracy_value = float(face_prediction[0][predicted_label_index] * 100)
print(predicted_label_index)
print(y_pred[predicted_label_index])

print(y_pred)
print(y_val)
# Display the image with the predicted label
img = cv2.imread(img_path)
cv2.putText(img, f"Predicted: {predicted_label_index}", (1, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

print(f"Prediction Label: ",predicted_label_index)

# Extract the numerical value of accuracy
print(f"Prediction Accuracy: {accuracy_value:.2f}%")

cv2.imshow('Image Detection', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


larine.jpg
1/1 [==============================] - 0s 27ms/step
[[0.00000000e+00 7.90133903e-10 2.27737498e-32 1.51735284e-26
  3.06580049e-28 2.52902467e-29 5.17768737e-24 1.25428895e-17
  2.00088431e-20 0.00000000e+00 1.71243102e-30 1.08197593e-29
  5.95248691e-23 1.64259882e-24 2.68269388e-27 9.87662420e-18
  4.63877493e-30 1.25943702e-19 1.05980570e-16 1.60764780e-32
  1.66483103e-37 1.00402445e-23 9.00291422e-36 6.46831183e-25
  1.81494515e-27 0.00000000e+00 3.52664865e-13 2.05269287e-33
  7.41355717e-01 5.31310883e-30 3.47352004e-26 1.50927552e-20
  3.24233071e-07 2.34245674e-14 5.68684084e-23 2.58643955e-01
  1.91350365e-32 6.53406761e-29 0.00000000e+00 7.53340897e-23
  4.50051967e-21]]
28
22
[32  1 20 10 11 11 12 12 13 13 14 14 15 15 16 16 17 17 18 18 19 19  2  2
 20 20 21 21 22 22 23 23 24 24 25 25 15 12 27 27 28 28 29 29  3  3 30 38
 31 31 32 32 33 33 34 34 35 35 36 36 37 37 38 38 39 39  4  4 40 17  5 12
  6  6  7  7  8  8  9  9]
[ 1 10 11 12 13 14 15 16 17 18 19  2 20 21 22 2

In [59]:
len(y_pred)
len(face_prediction[0])

41

In [23]:
import cv2
import numpy as np
from keras.preprocessing import image

# Load the pre-trained face cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load your pre-trained face recognition model
# (Assuming you have a model named 'face_model' loaded previously)

# Open a video capture object (0 represents the default camera, you can change it to the file path if using a video file)
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame using Haar Cascade
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the face region
        face_roi = gray[y:y+h, x:x+w]

        # Resize the face region to the required input size for your face recognition model
        face_roi_resized = cv2.resize(face_roi, (80, 70))

        # Convert to a format suitable for prediction
        img_array = image.img_to_array(face_roi_resized)
        img_array = np.expand_dims(img_array, axis=0)

        # Since the model expects grayscale images, discard color channels
        img_array = np.mean(img_array, axis=-1, keepdims=True)

        # Make predictions using your face recognition model
        face_prediction = face_model.predict(img_array)

        # Assuming face_model.predict returns a list of probabilities
        predicted_label_index = np.argmax(face_prediction)
        predicted_label = f"Label: {predicted_label_index}"

        # Get the accuracy value for the predicted label
        accuracy_value = float(face_prediction[0][predicted_label_index] * 100)
        accuracy_label = f"Accuracy: {accuracy_value:.2f}%"

        # If the accuracy is greater than 60, draw a rectangle around the face and display label and accuracy
        if accuracy_value > 60:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, predicted_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(frame, accuracy_label, (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Face Recognition', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 30ms/step
